In [9]:
import glob

# List all CSV files in the directory
files = glob.glob('/home/caow1/Downloads/datacuration/[0-9]*.csv')
print(files)  # Prints all CSV files in the Downloads directory


['/home/caow1/Downloads/datacuration/306289.csv', '/home/caow1/Downloads/datacuration/739689.csv', '/home/caow1/Downloads/datacuration/1118817.csv', '/home/caow1/Downloads/datacuration/69717.csv', '/home/caow1/Downloads/datacuration/371184.csv', '/home/caow1/Downloads/datacuration/638917.csv', '/home/caow1/Downloads/datacuration/1861970.csv', '/home/caow1/Downloads/datacuration/226585.csv', '/home/caow1/Downloads/datacuration/392130.csv', '/home/caow1/Downloads/datacuration/341717.csv', '/home/caow1/Downloads/datacuration/592966.csv', '/home/caow1/Downloads/datacuration/597116.csv', '/home/caow1/Downloads/datacuration/305799.csv', '/home/caow1/Downloads/datacuration/3367.csv', '/home/caow1/Downloads/datacuration/1718822.csv', '/home/caow1/Downloads/datacuration/81560.csv', '/home/caow1/Downloads/datacuration/1757236.csv', '/home/caow1/Downloads/datacuration/1485623.csv', '/home/caow1/Downloads/datacuration/68106.csv', '/home/caow1/Downloads/datacuration/722487.csv', '/home/caow1/Downlo

In [2]:
import pandas as pd 
import numpy as np 
import glob 

# the following combines all the CSV files and adds AID numbers to each respective CID
def combine():
  list_dfs = []
  for file_name in glob.glob('/home/caow1/Downloads/datacuration/[0-9]*.csv'):
      df = pd.read_csv(file_name, low_memory=False, index_col = False)
      df.drop(['PUBCHEM_RESULT_TAG', 'PUBCHEM_EXT_DATASOURCE_SMILES', 'PUBCHEM_ACTIVITY_URL',
      'PUBCHEM_ASSAYDATA_COMMENT'], axis = 1, inplace =True) # the author forgot a coma here, need to fix to run
      name = file_name[22:].split('.')[0] # change 33 to 23 since number is the 23rd char in dir
      df = df[df['PUBCHEM_CID'].notna()]
      aid_list = [] 
      for i in df.loc[:,'PUBCHEM_CID']:
          aid_list.append(int(name))
      df.insert(0, 'AID', aid_list, True) 
      df.PUBCHEM_SID = df.PUBCHEM_SID.astype(int)
      df.PUBCHEM_CID = df.PUBCHEM_CID.astype(int)
      list_dfs.append(df) 
  data = pd.concat(list_dfs, ignore_index = True)
  
  ## Uncomment below if you want to visualize the data. 
  data.to_csv('concatenated_gene_data.csv', index = False)
  return data 
combine()

,AID,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PubChem Standard Value,Standard Type,Standard Relation,Standard Value,Standard Units,...,Low Concentration (IC50 / 2),Qualifier,% Degradation,Kd Qualifier,Kd,qualifier,pKi,Activity Qualifier,EC50 Qualifier,EC50
0,306289,103454111,11357877,Unspecified,NaN,1,IC50,>,1000,nM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,306289,103531637,44430019,Active,NaN,0.088,IC50,=,88,nM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,306289,103549480,44440812,Unspecified,NaN,1,IC50,>,1000,nM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,306289,103549482,22055269,Active,NaN,0.43,IC50,=,430,nM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,306289,103549484,44440815,Active,NaN,0.11,IC50,=,110,nM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124975,1919643,483622424,146510002,Active,NaN,0.207,IC50,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124976,1919643,483622425,146448631,Active,NaN,0.208,IC50,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124977,1919643,483622426,146448756,Active,NaN,0.267,IC50,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124978,1919643,483622427,146448646,Active,NaN,0.048,IC50,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
import pandas as pd 
import numpy as np 
import glob 
def sorting():
    df = pd.read_csv('concatenated_gene_data.csv', low_memory =False)
    df = df.sort_values(by = ['PUBCHEM_CID', 'AID'], ascending = True, inplace = False)
    df.to_csv('sorted_concatenated_data.csv', index = False)
sorting()

In [34]:
import pandas as pd 
import numpy as np 
import glob 
def activity_defined():
    df = pd.read_csv('sorted_concatenated_data.csv', low_memory = False) # remember to take out the gene in file name
    IC50 = []
    select = []
    ki = []
    kd = []
    activity = []
    inhibit = []
    
    for i, j, z in zip(df['PubChem Standard Value'], df['Standard Type'], df['Standard Value']):
        if j == 'IC50':
            IC50.append(i)
        else:
            IC50.append(0)
        if j == 'Selectivity':
            select.append(z)
        else:
            select.append(0)
        if j == 'Inhibition':
            inhibit.append(z)
        else:
            inhibit.append(0)
        if j == 'Ki':
            ki.append(i)
        else:
            ki.append(0)
        if j == 'Kd':
            kd.append(i)
        else:
            kd.append(0)
        if j == 'Activity':
            activity.append(z)
        else:
            activity.append(0)
    df.insert(9, 'IC50 (mm)', IC50, True)
    df.insert(10, 'Selectivity', select, True)
    df.insert(11, 'Inhibition (%)', inhibit, True)
    df.insert(12, 'Ki (mm)', ki, True)
    df.insert(13, 'Kd (mm)', kd, True)
    df.insert(14, 'Activity (%)', activity, True)
    df.to_csv('parameters_defined_concatenated_data.csv', index = False)
    return df 
activity_defined()

,AID,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PubChem Standard Value,Standard Type,Standard Relation,Standard Value,IC50 (mm),...,Low Concentration (IC50 / 2),Qualifier,% Degradation,Kd Qualifier,Kd,qualifier,pKi,Activity Qualifier,EC50 Qualifier,EC50
0,625186,103601826,174,Unspecified,NaN,NaN,IC50,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,625186,103601826,174,Unspecified,NaN,NaN,Ki,NaN,NaN,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,613155,103517679,190,Unspecified,NaN,NaN,Inhibition,=,3.0,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,625186,103165858,237,Unspecified,NaN,NaN,IC50,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,625186,103165858,237,Unspecified,NaN,NaN,Ki,NaN,NaN,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124975,1927648,494416958,171355565,Active,NaN,0.0210,IC50,=,21.0,0.021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124976,1923077,494418062,171356152,Unspecified,NaN,10.0000,IC50,>,10000.0,10.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124977,1927648,494418971,171356652,Active,NaN,0.0210,IC50,=,21.0,0.021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124978,1927648,494419010,171356677,Active,NaN,0.0420,IC50,=,42.0,0.042,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
import pandas as pd
import requests #important! 

def parsing():
    result_dict = {}
    na = []
    df = pd.read_csv('parameters_defined_concatenated_data.csv', low_memory=False)
    aid_list = list(df['AID'].unique())
    for i in aid_list:
        r = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{i}/summary/JSON")
        data = r.json()
        for key, value in data['AssaySummaries'].items():
            for adict in value:
                try:
                    if adict['Comment'][0].find('10uM') != -1:
                        result_dict[i] = [adict['Comment'][0]]
                    for element in adict['Comment']:
                        if element.find('erbB-2') != -1:
                            result_dict[i] += ['erbB-2']
                except:
                    na.append(i)
    activity = []
    target = []
    for i in df.loc[:, 'AID']:
        if i in result_dict.keys():
            activity.append(result_dict[i][0])
            try:
                target.append(result_dict[i][1])
            except:
                target.append('check')
        else:
            target.append('need to check')
            activity.append('unknown')
    #print(activity)
    #print(len(activity))
    #print(len(df['AID']))
    
    df.insert(17, 'target_name', target, True)
    df.insert(5, 'activity_added', activity, True)
    df.to_csv('target_added_2064.csv', index=False)
    return df        
parsing()

,AID,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,activity_added,PubChem Standard Value,Standard Type,Standard Relation,Standard Value,...,Low Concentration (IC50 / 2),Qualifier,% Degradation,Kd Qualifier,Kd,qualifier,pKi,Activity Qualifier,EC50 Qualifier,EC50
0,625186,103601826,174,Unspecified,NaN,Compounds with activity <= 10uM or explicitly ...,NaN,IC50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,625186,103601826,174,Unspecified,NaN,Compounds with activity <= 10uM or explicitly ...,NaN,Ki,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,613155,103517679,190,Unspecified,NaN,unknown,NaN,Inhibition,=,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,625186,103165858,237,Unspecified,NaN,Compounds with activity <= 10uM or explicitly ...,NaN,IC50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,625186,103165858,237,Unspecified,NaN,Compounds with activity <= 10uM or explicitly ...,NaN,Ki,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124975,1927648,494416958,171355565,Active,NaN,Compounds with activity <= 10uM or explicitly ...,0.0210,IC50,=,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124976,1923077,494418062,171356152,Unspecified,NaN,unknown,10.0000,IC50,>,10000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124977,1927648,494418971,171356652,Active,NaN,Compounds with activity <= 10uM or explicitly ...,0.0210,IC50,=,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124978,1927648,494419010,171356677,Active,NaN,Compounds with activity <= 10uM or explicitly ...,0.0420,IC50,=,42.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
def add_filter():
    df = pd.read_csv('target_added_2064.csv', low_memory = False)
    df = df[(df['Target Accession(s)'] == 'P04626') | (df['Target'].str.contains('erbB-2', na=False))| 
            (df['Target'].str.contains('Her2', na=False))| (df['Panel Name']== 'ERBB2') | (df['Panel Target'] == 2064)]
    df.to_csv('add_filtering_target_2064.csv', index = False)
    return df
add_filter()

def specified():
    df = pd.read_csv('add_filtering_target_2064.csv', low_memory = False)
    df = df.sort_values(by= ['AID'], inplace = False)
    active_df = df[df['PUBCHEM_ACTIVITY_OUTCOME'] == 'Active']
    inactive_df = df[df['PUBCHEM_ACTIVITY_OUTCOME'] == 'Inactive']
    active_df.to_csv('actives.csv', index = False) 
    inactive_df.to_csv('inactive.csv', index = False)
    return active_df 
specified()

,AID,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,activity_added,PubChem Standard Value,Standard Type,Standard Relation,Standard Value,...,Low Concentration (IC50 / 2),Qualifier,% Degradation,Kd Qualifier,Kd,qualifier,pKi,Activity Qualifier,EC50 Qualifier,EC50
0,493040,103904783,1400,Active,NaN,unknown,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.2,NaN,NaN,NaN
540,493040,103904314,49830359,Active,NaN,unknown,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.9,NaN,NaN,NaN
559,493040,103904402,49830398,Active,NaN,unknown,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.9,NaN,NaN,NaN
538,493040,103904310,49830357,Active,NaN,unknown,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.9,NaN,NaN,NaN
537,493040,103904309,49830356,Active,NaN,unknown,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2483,1920053,486503916,167318766,Active,NaN,"Compounds with any of Ki, IC50, Kd, or EC50 ac...",0.0638,IC50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2075,1920053,486503647,166928149,Active,NaN,"Compounds with any of Ki, IC50, Kd, or EC50 ac...",0.1500,IC50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2485,1920053,486503528,167318767,Active,NaN,"Compounds with any of Ki, IC50, Kd, or EC50 ac...",0.4030,IC50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2073,1920053,486503638,166928148,Active,NaN,"Compounds with any of Ki, IC50, Kd, or EC50 ac...",0.1130,IC50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
def type_added():
    result_dict = {}
    types = []
    df = pd.read_csv('actives.csv', low_memory=False)
    aid_list = list(df['AID'])
    for i in aid_list:
        r = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{i}/summary/JSON")
        data = r.json()
        for key, value in data['AssaySummaries'].items():
            for adict in value:
                try:
                    types.append(adict['Method'])
                except:
                    types.append('n/a')
    df.insert(4, 'AID_type', types, True)combine
    #df.sort_values(by= ['PUBCHEM_CID', 'AID'], inplace = True)
    df.to_csv('actives_ordered_labeled.csv', index = False)
    return types
type_added()

['other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',
 'confirmatory',


In [40]:
def type_added():
    result_dict = {}
    types = []
    df = pd.read_csv('inactive.csv', low_memory=False) #add confirmatory or not for the inactive file 
    aid_list = list(df['AID'])
    for i in aid_list:
        r = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{i}/summary/JSON")
        data = r.json()
        for key, value in data['AssaySummaries'].items():
            for adict in value:
                try:
                    types.append(adict['Method'])
                except:
                    types.append('n/a')
    df.insert(4, 'AID_type', types, True)
    #df.sort_values(by= ['PUBCHEM_CID', 'AID'], inplace = True)
    df.to_csv('inactive_ordered_labeled.csv', index = False)
    return types
type_added()

['other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',


In [41]:
def confirmatory_only():
    df = pd.read_csv('actives_ordered_labeled.csv', low_memory = False)
    df = df[df['AID_type'] == 'confirmatory']
    df = df[(df['target_name'] != 'EGFR') & (df['target_name'] != 'EGFR mut')]
    #df.to_csv('confirmatory_actives.csv', index = False)
    df.PUBCHEM_SID = df.PUBCHEM_SID.astype(int)
    grouped = df.groupby(['PUBCHEM_CID', 'AID'], as_index = False).mean()
    grouped.to_csv('grouped_confirmatory_actives.csv', index = False)
    return grouped.columns
confirmatory_only()

Index(['PUBCHEM_CID', 'AID', 'PUBCHEM_SID', 'PUBCHEM_ACTIVITY_SCORE',
       'PubChem Standard Value', 'Standard Value', 'IC50 (mm)', 'Selectivity',
       'Inhibition (%)', 'Ki (mm)', 'Kd (mm)', 'Activity (%)',
       'Standard Units', 'Activity Comment', 'Data Validity Comment',
       'pKd_min', 'pKd_max', 'pIC50_min', 'pIC50_max', 'Type', 'Action',
       'Reference (PubMed ID)', 'IC50', 'Enzyme Activity [1]',
       'Enzyme Activity [2]', 'Average Enzyme Activity', 'Panel ID',
       'Panel Target', '%Inhibition', 'High Concentration (5 x IC50)',
       'Low Concentration (IC50 / 2)', 'Qualifier', '% Degradation',
       'Kd Qualifier', 'Kd', 'qualifier', 'pKi', 'Activity Qualifier',
       'EC50 Qualifier', 'EC50'],
      dtype='object')

In [42]:
def confirmatory_only():
    df = pd.read_csv('inactive_ordered_labeled.csv', low_memory = False)
    df = df[df['AID_type'] == 'confirmatory']
    df = df[(df['target_name'] != 'EGFR') & (df['target_name'] != 'EGFR mut')]
    #df.to_csv('confirmatory_actives.csv', index = False)
    df.PUBCHEM_SID = df.PUBCHEM_SID.astype(int)
    grouped = df.groupby(['PUBCHEM_CID', 'AID'], as_index = False).mean()
    grouped.to_csv('grouped_confirmatory_inactive.csv', index = False)
    return grouped.columns
confirmatory_only()

Index(['PUBCHEM_CID', 'AID', 'PUBCHEM_SID', 'PUBCHEM_ACTIVITY_SCORE',
       'PubChem Standard Value', 'Standard Value', 'IC50 (mm)', 'Selectivity',
       'Inhibition (%)', 'Ki (mm)', 'Kd (mm)', 'Activity (%)',
       'Standard Units', 'Activity Comment', 'Data Validity Comment',
       'pKd_min', 'pKd_max', 'pIC50_min', 'pIC50_max', 'Type', 'Action',
       'Reference (PubMed ID)', 'IC50', 'Enzyme Activity [1]',
       'Enzyme Activity [2]', 'Average Enzyme Activity', 'Panel ID',
       '%Inhibition', 'High Concentration (5 x IC50)',
       'Low Concentration (IC50 / 2)', 'Qualifier', '% Degradation',
       'Kd Qualifier', 'Kd', 'pKi', 'Activity Qualifier', 'EC50 Qualifier',
       'EC50'],
      dtype='object')

In [43]:
def mw():
    mw = []
    df = pd.read_csv('grouped_confirmatory_actives.csv', low_memory = False)
    dist_CIDs = list(df['PUBCHEM_CID'])
    for y in dist_CIDs:
        r = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{y}/property/MolecularWeight/JSON')
        data = r.json()
        for key, value in data.items():
            try:
                mw.append(str(value['Properties'][0]['MolecularWeight']))
            except:
                mw.append('n/a')
    df.insert(5, 'MolecularWeight', mw, True)
    df.to_csv('mw_added_grouped_actives.csv', index = False)
    return mw
mw()

['381.2',
 '369.2',
 '466.5',
 '446.9',
 '393.4',
 '393.4',
 '581.1',
 '581.1',
 '581.1',
 '581.1',
 '464.8',
 '424.3',
 '452.3',
 '480.4',
 '452.3',
 '480.4',
 '450.3',
 '464.4',
 '466.3',
 '482.4',
 'n/a',
 'n/a',
 '411.3',
 '455.3',
 '441.3',
 '397.2',
 '426.3',
 '454.4',
 '482.4',
 '466.4',
 '468.3',
 '484.4',
 '496.4',
 '496.4',
 '470.4',
 '456.3',
 '549.5',
 '413.3',
 '438.3',
 '468.3',
 '454.3',
 '496.4',
 '492.0',
 '545.7',
 '569.7',
 '613.7',
 '580.1',
 '598.1',
 '343.4',
 '616.7',
 '520.6',
 '357.4',
 '383.4',
 '506.6',
 '520.6',
 '510.5',
 '438.5',
 '467.0',
 '554.7',
 '573.1',
 '562.7',
 '465.4',
 '512.4',
 '557.0',
 '557.0',
 '557.0',
 '557.0',
 '549.9',
 '599.9',
 '473.4',
 '466.3',
 '468.3',
 '473.4',
 '474.9',
 '522.8',
 '563.8',
 '485.9',
 '485.9',
 '459.4',
 '535.8',
 '409.9',
 '427.3',
 '459.4',
 '540.8',
 '544.3',
 '562.3',
 '492.7',
 '535.8',
 '540.8',
 '433.4',
 '512.8',
 '519.4',
 '540.8',
 '552.3',
 '478.3',
 '502.4',
 '433.3',
 '466.3',
 '491.4',
 '491.4',
 '54

In [2]:
import pandas as pd
import requests #important! 
import numpy as np
def mw():
    mw = []
    df = pd.read_csv('grouped_confirmatory_inactive.csv', low_memory = False)
    dist_CIDs = list(df['PUBCHEM_CID'])
    for y in dist_CIDs:
        r = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{y}/property/MolecularWeight/JSON')
        data = r.json()
        for key, value in data.items():
            try:
                mw.append(str(value['Properties'][0]['MolecularWeight']))
            except:
                mw.append('n/a')
    df.insert(5, 'MolecularWeight', mw, True)
    df.to_csv('mw_added_grouped_inactive.csv', index = False)
    return mw
mw()

['449.3',
 '383.2',
 '383.2',
 '412.3',
 '383.2',
 '473.4',
 '459.4',
 '557.2',
 '533.5',
 '479.9',
 '577.9',
 '478.3',
 '520.4',
 '412.7',
 '428.3',
 '346.6',
 '448.3',
 '495.3',
 '403.8',
 '493.4',
 '506.4',
 '462.3',
 '516.4',
 '515.4',
 '501.4',
 '535.4',
 '516.8',
 '544.5',
 '487.3',
 '491.4',
 '510.3',
 '554.8',
 '523.4',
 '554.3',
 '503.4',
 '530.5',
 '517.4',
 '521.4',
 '493.4',
 '503.4',
 '517.4',
 '516.4',
 '516.4',
 '516.4',
 '529.4',
 '491.4',
 'n/a',
 '515.4',
 '530.5',
 '445.9',
 '285.34',
 '785.8',
 '765.9',
 '953.5',
 '527.7',
 '430.5',
 '482.5',
 '738.2',
 '694.1',
 '909.5',
 '970.2']

In [3]:
def below_800():
    df = pd.read_csv('mw_added_grouped_actives.csv', low_memory = False)
    df = df[df['MolecularWeight'] <= 800]
    df.to_csv('below_800_mw_grouped_actives.csv', index = False)
    return df
below_800()

,PUBCHEM_CID,AID,PUBCHEM_SID,PUBCHEM_ACTIVITY_SCORE,PubChem Standard Value,MolecularWeight,Standard Value,IC50 (mm),Selectivity,Inhibition (%),...,Low Concentration (IC50 / 2),Qualifier,% Degradation,Kd Qualifier,Kd,qualifier,pKi,Activity Qualifier,EC50 Qualifier,EC50
0,2776,1795646,8034229.0,NaN,0.3780,381.2,NaN,0.3780,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4708,1795646,8034230.0,NaN,0.1140,369.2,NaN,0.1140,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5279,1798359,225157243.0,NaN,0.0630,466.5,NaN,0.0630,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,123631,1795774,8035066.0,NaN,0.2400,446.9,NaN,0.2400,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,176870,1795774,8035065.0,NaN,0.7600,393.4,NaN,0.7600,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,169552774,1920053,486503829.0,NaN,0.0321,482.5,NaN,0.0321,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2141,169552775,1920052,486503871.0,NaN,0.0166,498.9,NaN,0.0166,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2142,169552775,1920053,486503871.0,NaN,0.0323,498.9,NaN,0.0323,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2143,170776255,1798142,464922092.0,NaN,0.3030,588.1,NaN,0.3030,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def below_800():
    df = pd.read_csv('mw_added_grouped_inactive.csv', low_memory = False)
    df = df[df['MolecularWeight'] <= 800]
    df.to_csv('below_800_mw_grouped_inactive.csv', index = False)
    return df
below_800()

,PUBCHEM_CID,AID,PUBCHEM_SID,PUBCHEM_ACTIVITY_SCORE,PubChem Standard Value,MolecularWeight,Standard Value,IC50 (mm),Selectivity,Inhibition (%),...,High Concentration (5 x IC50),Low Concentration (IC50 / 2),Qualifier,% Degradation,Kd Qualifier,Kd,pKi,Activity Qualifier,EC50 Qualifier,EC50
0,5328980,1795646,8034257,NaN,15.624,449.30,NaN,15.624,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5328981,1795646,8034258,NaN,15.238,383.20,NaN,15.238,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5328982,1795646,8034259,NaN,91.325,383.20,NaN,91.325,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5328986,1795646,8034263,NaN,25.710,412.30,NaN,25.710,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5328988,1795646,8034265,NaN,21.657,383.20,NaN,21.657,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10005056,1797499,386053048,NaN,10.500,473.40,NaN,10.500,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10253661,1797499,386053049,NaN,10.500,459.40,NaN,10.500,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10347695,1797499,386053056,NaN,10.500,557.20,NaN,10.500,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10369818,1797499,386053007,NaN,10.500,533.50,NaN,10.500,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10390394,1797499,386053058,NaN,10.500,479.90,NaN,10.500,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def sdf():
    df = pd.read_csv('below_800_mw_grouped_actives.csv', low_memory = False)
    dist_CIDs = list(df['PUBCHEM_CID'].unique())
    for i in dist_CIDs:
        r = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{i}/SDF')
        if r.status_code == 200:
            with open(f'{i}' + '.sdf', 'wb') as file:
                file.write(r.content)
            print('File download successful')
            
        else:
            print('Error with', f'{i}')
sdf()

File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
Error with 5328957
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
Error with 5329477
File download successful
File downloa

In [6]:
def sdf():
    df = pd.read_csv('below_800_mw_grouped_inactive.csv', low_memory = False)
    dist_CIDs = list(df['PUBCHEM_CID'].unique())
    for i in dist_CIDs:
        r = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{i}/SDF')
        if r.status_code == 200:
            with open(f'{i}' + '.sdf', 'wb') as file:
                file.write(r.content)
            print('File download successful')
            
        else:
            print('Error with', f'{i}')
sdf()

File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful


In [7]:
import pandas as pd 
import numpy as np 
import glob 
def below_600():
    df = pd.read_csv('mw_added_grouped_actives.csv', low_memory = False)
    df = df[df['MolecularWeight'] <= 600]
    df.to_csv('below_600_mw_grouped_actives.csv', index = False)
    return df
below_600()

,PUBCHEM_CID,AID,PUBCHEM_SID,PUBCHEM_ACTIVITY_SCORE,PubChem Standard Value,MolecularWeight,Standard Value,IC50 (mm),Selectivity,Inhibition (%),...,Low Concentration (IC50 / 2),Qualifier,% Degradation,Kd Qualifier,Kd,qualifier,pKi,Activity Qualifier,EC50 Qualifier,EC50
0,2776,1795646,8034229.0,NaN,0.3780,381.2,NaN,0.3780,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4708,1795646,8034230.0,NaN,0.1140,369.2,NaN,0.1140,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5279,1798359,225157243.0,NaN,0.0630,466.5,NaN,0.0630,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,123631,1795774,8035066.0,NaN,0.2400,446.9,NaN,0.2400,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,176870,1795774,8035065.0,NaN,0.7600,393.4,NaN,0.7600,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,169552774,1920052,486503829.0,NaN,0.0263,482.5,NaN,0.0263,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2140,169552774,1920053,486503829.0,NaN,0.0321,482.5,NaN,0.0321,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2141,169552775,1920052,486503871.0,NaN,0.0166,498.9,NaN,0.0166,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2142,169552775,1920053,486503871.0,NaN,0.0323,498.9,NaN,0.0323,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
## This block gets rid of the duplicate CIDs and makes sure each CID is unique 

# Define the file paths
input_file = 'below_600_mw_grouped_actives.csv'
output_file = 'below_600_mw_grouped_actives_unique.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Check if the 'PUBCHEM_CID' column exists
if 'PUBCHEM_CID' not in df.columns:
    raise ValueError("The column 'PUBCHEM_CID' does not exist in the CSV file.")

# Display the number of duplicates before removing
duplicates_before = df['PUBCHEM_CID'].duplicated().sum()
print(f"Number of duplicates before removing: {duplicates_before}")

# Remove duplicate rows based on 'PUBCHEM_CID' column
df_unique = df.drop_duplicates(subset='PUBCHEM_CID', keep='first')

# Display the number of duplicates after removing
duplicates_after = df_unique['PUBCHEM_CID'].duplicated().sum()
print(f"Number of duplicates after removing: {duplicates_after}")

# Save the DataFrame with unique 'PUBCHEM_CID' values to a new CSV file
df_unique.to_csv(output_file, index=False)

print(f"Unique data saved to '{output_file}'")


Number of duplicates before removing: 616
Number of duplicates after removing: 0
Unique data saved to 'below_600_mw_grouped_actives_unique.csv'


In [9]:
## This block outputs a file that contains all AIDs that have more than 50 CIDS associated 
## Each AID represents an assay and each CID represents a compound 

import pandas as pd

# Define the file paths
input_file = 'below_600_mw_grouped_actives.csv'
output_file = 'aid_with_more_than_50_unique_cids.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Check if the required columns exist
required_columns = ['AID', 'PUBCHEM_CID']
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"The column '{col}' does not exist in the CSV file.")

# Group by 'AID' and count unique 'PUBCHEM_CID' values
aid_counts = df.groupby('AID')['PUBCHEM_CID'].nunique()

# Filter AID values with more than 50 unique PUBCHEM_CID values
aid_filtered = aid_counts[aid_counts > 50].reset_index()

# Rename the columns for clarity
aid_filtered.columns = ['AID', 'Unique_CID_Count']

# Save the results to a new CSV file
aid_filtered.to_csv(output_file, index=False)

print(f"Results saved to '{output_file}'")


Results saved to 'aid_with_more_than_50_unique_cids.csv'


In [11]:
##a high IC50 means more conc of the molecule is needed thus drug less potent
##this block screens out the ones with IC50 in mm thats greater than 1 


import pandas as pd

# Define the file paths
input_file = 'below_600_mw_grouped_actives_unique.csv'
output_file = 'below_600_mw_IC50_less1_actives_unique.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Check if the required column exists
if 'IC50 (mm)' not in df.columns:
    raise ValueError("The column 'IC50 (mm)' does not exist in the CSV file.")

filtered_df = df[df['IC50 (mm)'] <= 1]

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv(output_file, index=False)

print(f"Filtered results saved to '{output_file}'")


Filtered results saved to 'below_600_mw_IC50_less1_actives_unique.csv'


In [12]:

def sdf():
    df = pd.read_csv('below_600_mw_IC50_less1_actives_unique.csv', low_memory = False)
    dist_CIDs = list(df['PUBCHEM_CID'].unique())
    for i in dist_CIDs:
        r = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{i}/SDF')
        if r.status_code == 200:
            with open(f'{i}' + '.sdf', 'wb') as file:
                file.write(r.content)
            print('File download successful')
            
        else:
            print('Error with', f'{i}')
sdf()

File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful


In [1]:
## THis block outputs all of the unspecified in the target_added_2064.csv file to another file called unspecified groups, 
## it also includes their AID and the count of unspecified ones within that AID

import pandas as pd

# Define the file paths
input_file = 'target_added_2064.csv'
output_file = 'unspecified_groups.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Check if the required columns exist
required_columns = ['AID', 'PUBCHEM_ACTIVITY_OUTCOME']
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"The column '{col}' does not exist in the CSV file.")

# Filter rows where 'PUBCHEM_ACTIVITY_OUTCOME' is 'Unspecified'
filtered_df = df[df['PUBCHEM_ACTIVITY_OUTCOME'] == 'Unspecified']

# Group by 'AID' and count the number of 'Unspecified' outcomes
aid_counts = filtered_df.groupby('AID').size().reset_index(name='Unspecified_Count')

# Save the results to a new CSV file
aid_counts.to_csv(output_file, index=False)

print(f"Results saved to '{output_file}'")


Results saved to 'unspecified_groups.csv'


/home/mamuna/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (7,8,16,17,19,24,25,26,28,29,30,31,37,41,43,47,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
import pandas as pd

# Define the file paths
input_file = 'target_added_2064.csv'
output_file = 'inactive_unspecified_merged.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Check if the required column exists
if 'PUBCHEM_ACTIVITY_OUTCOME' not in df.columns:
    raise ValueError("The column 'PUBCHEM_ACTIVITY_OUTCOME' does not exist in the CSV file.")

# Filter rows where 'PUBCHEM_ACTIVITY_OUTCOME' is either 'Inactive' or 'Unspecified'
filtered_df = df[df['PUBCHEM_ACTIVITY_OUTCOME'].isin(['Inactive', 'Unspecified'])]

# Save the results to a new CSV file
filtered_df.to_csv(output_file, index=False)

print(f"Results saved to '{output_file}'")


/home/mamuna/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (7,8,16,17,19,24,25,26,28,29,30,31,37,41,43,47,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Results saved to 'inactive_unspecified_merged.csv'


In [3]:
## This block gets rid of the duplicate CIDs and makes sure each CID is unique 
import pandas as pd
import numpy as np 
import glob 

# Define the file paths
input_file = 'inactive_unspecified_merged.csv'
output_file = 'inactive_unique.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Check if the 'PUBCHEM_CID' column exists
if 'PUBCHEM_CID' not in df.columns:
    raise ValueError("The column 'PUBCHEM_CID' does not exist in the CSV file.")

# Display the number of duplicates before removing
duplicates_before = df['PUBCHEM_CID'].duplicated().sum()
print(f"Number of duplicates before removing: {duplicates_before}")

# Remove duplicate rows based on 'PUBCHEM_CID' column
df_unique = df.drop_duplicates(subset='PUBCHEM_CID', keep='first')

# Display the number of duplicates after removing
duplicates_after = df_unique['PUBCHEM_CID'].duplicated().sum()
print(f"Number of duplicates after removing: {duplicates_after}")

# Save the DataFrame with unique 'PUBCHEM_CID' values to a new CSV file
df_unique.to_csv(output_file, index=False)

print(f"Unique data saved to '{output_file}'")

Number of duplicates before removing: 73482
Number of duplicates after removing: 0
Unique data saved to 'inactive_unique.csv'


/home/mamuna/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (7,8,16,17,19,28,29,30,31,37,41,43,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
# download all of the unique inactive ones for training dataset later on

import requests #important! 
def sdf():
    df = pd.read_csv('inactive_unique.csv', low_memory = False)
    dist_CIDs = list(df['PUBCHEM_CID'].unique())
    for i in dist_CIDs:
        r = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{i}/SDF')
        if r.status_code == 200:
            with open(f'{i}' + '.sdf', 'wb') as file:
                file.write(r.content)
            print('File download successful')
            
        else:
            print('Error with', f'{i}')
sdf()

File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
Error with 757
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
Error with 1057
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download successful
File download succe